# DASK to ServiceX

In this demo we'll take advantage of DASK and ServiceX.

## Assumptions:

* We don't start anything until we know the number of files that SX will produce
* We are ok with some files failing coming out of SX
* We are going to do one partition per file
* When we start we don't necessarily know all the files produced.

## Design Outline

* A single `dask` task/layer that has a single output per partition. The output is just a string.
* The `AwkwardInputLayer` that has looks at the task input and loads that data from `minio`

This version of things will work only for local files - once this works we can move it to a SX prototype.

This was written before any code below.

## Research

A few things to help get started below.

### How does the current uproot/dask work?

The `from_map` is the base of how `dask_awkward` builds the production. One very nice thing - this is a public interface, so we
can rely on it in libraries. 

```python
def from_map(
    func: Callable,
    *iterables: Iterable,
    args: tuple[Any, ...] | None = None,
    label: str | None = None,
    token: str | None = None,
    divisions: tuple[int, ...] | tuple[None, ...] | None = None,
    meta: ak.Array | None = None,
    **kwargs: Any,
) -> Array | tuple[Array, Array]:
    """Create an Array collection from a custom mapping.

    Parameters
    ----------
    func : Callable
        Function used to create each partition.
    *iterables : Iterable
        Iterable objects to map to each output partition. All
        iterables must be the same length. This length determines the
        number of partitions in the output collection (only one
        element of each iterable will be passed to `func` for each
        partition).
    args : tuple
        Tuple of positional arguments to append after mapped arguments.
    label : str, optional
        String to use as the function-name label in the output
        collection-key names.
    token : str, optional
        String to use as the "token" in the output collection-key names.
    divisions : tuple[int, ...] | tuple[None, ...], optional
        Partition boundaries (if known).
    meta : Array, optional
        Collection metadata array, if known (the awkward-array type
        tracer)
    **kwargs : Any
        Keyword arguments passed to `func`.

    Returns
    -------
    Array
        Array collection.

    """

    if not callable(func):
        raise ValueError("`func` argument must be `callable`")
    lengths = set()
    iters: list[Iterable] = list(iterables)
    for i, iterable in enumerate(iters):
        if not isinstance(iterable, Iterable):
            raise ValueError(
                f"All elements of `iterables` must be Iterable, got {type(iterable)}"
            )
        try:
            lengths.add(len(iterable))  # type: ignore
        except (AttributeError, TypeError):
            iters[i] = list(iterable)
            lengths.add(len(iters[i]))  # type: ignore
    if len(lengths) == 0:
        raise ValueError("`from_map` requires at least one Iterable input")
    elif len(lengths) > 1:
        raise ValueError("All `iterables` must have the same length")
    if lengths == {0}:
        raise ValueError("All `iterables` must have a non-zero length")

    # Check for `produces_tasks` and `creation_info`
    produces_tasks = kwargs.pop("produces_tasks", False)
    # creation_info = kwargs.pop("creation_info", None)

    if produces_tasks or len(iters) == 1:
        if len(iters) > 1:
            # Tasks are not detected correctly when they are "packed"
            # within an outer list/tuple
            raise ValueError(
                "Multiple iterables not supported when produces_tasks=True"
            )
        inputs = list(iters[0])
        packed = False
    else:
        # Structure inputs such that the tuple of arguments pair each 0th,
        # 1st, 2nd, ... elements together; for example:
        # from_map(f, [1, 2, 3], [4, 5, 6]) --> [f(1, 4), f(2, 5), f(3, 6)]
        inputs = list(zip(*iters))
        packed = True

    # Define collection name
    label = label or funcname(func)
    token = token or tokenize(func, iters, meta, **kwargs)
    name = f"{label}-{token}"

    # Define io_func

    # FIXME: projection etc.
    if packed or args or kwargs:
        func = PackedArgCallable(
            func,
            args=args,
            kwargs=kwargs,
            packed=packed,
        )

    # Special `io_func` implementations can implement mocking and optionally
    # support buffer projection.
    if io_func_implements_mocking(func):
        io_func = func
        array_meta = cast(ImplementsMocking, func).mock()
    # If we know the meta, we can spoof mocking
    elif meta is not None:
        io_func = IOFunctionWithMocking(meta, func)
        array_meta = meta
    # Without `meta`, the meta will be computed by executing the graph
    else:
        io_func = func
        array_meta = None

    dsk = AwkwardInputLayer(name=name, inputs=inputs, io_func=io_func)

    hlg = HighLevelGraph.from_collections(name, dsk)
    if divisions is not None:
        result = new_array_object(hlg, name, meta=array_meta, divisions=divisions)
    else:
        result = new_array_object(hlg, name, meta=array_meta, npartitions=len(inputs))

    if io_func_implements_report(io_func):
        if cast(ImplementsReport, io_func).return_report:
            res = result.map_partitions(first, meta=array_meta, output_divisions=1)
            rep = result.map_partitions(second, meta=empty_typetracer())
            return res, rep

    return result
```

## Imports

In [1]:
import dask_awkward as dak
import awkward as ak
import dask
import uproot

from dask.highlevelgraph import Layer, HighLevelGraph
from dask.distributed import Client

client = Client()

## SX Support

The code below belongs in the `sx-awk` library.

This is the ServiceX layer. It is responsible for all communication with ServiceX, and finding the files (and URL's) from `minio`.

In [2]:
from collections.abc import Set
from typing import AbstractSet, Any, Dict, Iterator, KeysView, List
import logging

class SXLayer(Layer):
    def __init__(self, sx_query_guid):
        super().__init__()
        self._query_guid = sx_query_guid

        # Create a task that will be executed when the layer is computed,
        # and will fetch the list of files from SX.
        k = f"SX-query-{self._query_guid}"
        self._tasks: Dict[str, Any] = {k: dask.delayed(self._fetch_files, name=k)}

    def _fetch_files(self) -> str:
        # This is where the actual fetching of the files from SX would happen.
        # For now, just return a list of files.
        logging.warn("Returning a file")
        return "0fc6e51a5ea6dea107c195591d20a1b2-15.26710677._000019.pool.root.1"

    def __getitem__(self, __key) -> Any:
        return self._tasks[str(__key)]
    
    def keys(self):
        return self._tasks.keys()
    
    def __len__(self) -> int:
        return len(self._tasks)
    
    def get_output_keys(self) -> AbstractSet[str]:
        return {f"SX-query-{self._query_guid}"}

    def __iter__(self):
        return iter(self._tasks)
    
    def is_materialized(self) -> bool:
        return False

In [3]:
l = SXLayer("182382781")
hlg = HighLevelGraph(
    layers={"l1": l},
    dependencies={},
)

In [4]:
r = client.compute(hlg)

In [5]:
type(r)


dask.highlevelgraph.HighLevelGraph

## The `uproot.dask` way

This is a very simple call - here for reference.

In [31]:
filename = "0fc6e51a5ea6dea107c195591d20a1b2-15.26710677._000019.pool.root.1"
ar = uproot.dask({filename: "treeme"}, open_files=False)

In [32]:
pt = ar.JetPt * 5

And lets look at the layers/etc. for this for reference.

In [34]:
pt.compute()

<Array [[509, 315], [...], ..., [763, 452, 270]] type='50000 * var * float64'>

In [30]:
graph = pt.__dask_graph__()
graph

HighLevelGraph with 3 layers.
<dask.highlevelgraph.HighLevelGraph object at 0x228a09129d0>
 0. from-uproot-e52447bb623d6bb74ea637be420b3a92
 1. JetPt-db1ead26b17df9bb31ff1f87da0b62e1
 2. multiply-78f1358d2f645d9af93f24ae88dec8fb

Lets look at the input layer here

In [48]:
from_uproot_key = [k for k in graph.layers.keys() if k.startswith("from")][0]
print(f"From uproot key: {from_uproot_key}")
from_uproot_first_output = list(graph.layers[from_uproot_key].keys())[0]
print(f"From uproot first output: {from_uproot_first_output}")
print(f"The function that gets executed and the arguments: {graph.layers[from_uproot_key][from_uproot_first_output]}")

From uproot key: from-uproot-e52447bb623d6bb74ea637be420b3a92
From uproot first output: ('from-uproot-e52447bb623d6bb74ea637be420b3a92', 0)
The function that gets executed and the arguments: (subgraph_callable-00b359c2-04df-41ea-a368-1c60e311ce07, ('0fc6e51a5ea6dea107c195591d20a1b2-15.26710677._000019.pool.root.1', 'treeme', 0, 1, False))


2024-02-22 21:55:06,570 - distributed.scheduler - WARNING - Worker failed to heartbeat within 300 seconds. Closing: <WorkerState 'tcp://127.0.0.1:62130', name: 1, status: running, memory: 0, processing: 0>
2024-02-22 21:55:06,580 - distributed.scheduler - WARNING - Worker failed to heartbeat within 300 seconds. Closing: <WorkerState 'tcp://127.0.0.1:62136', name: 2, status: running, memory: 0, processing: 0>
2024-02-22 21:55:06,586 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:62130'.
2024-02-22 21:55:06,589 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:62136'.
2024-02-22 23:31:35,542 - distributed.scheduler - WARNING - Worker failed to heartbeat within 300 seconds. Closing: <WorkerState 'tcp://127.0.0.1:62131', name: 0, status: running, memory: 0, processing: 0>
2024-02-22 23:31:35,561 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:621

We want the `('0fc6e51a5ea6dea107c195591d20a1b2-15.26710677._000019.pool.root.1', 'treeme', 0, 1, False)` to be the argument/output from a previous run.

## Toy Demo

Used AI to come up with this (minus a few syntax errors and missing the `Client.get`). This shows how to build everything from scratch.

In [38]:
from collections.abc import Set
import typing


class CustomLayer(Layer):
    def __init__(self, name, dependencies, tasks):
        super().__init__()
        self.name = name
        self.dependencies = dependencies
        self.tasks = tasks

    def __getitem__(self, key):
        return self.tasks[key]

    def __iter__(self):
        return iter(self.tasks)

    def __len__(self):
        return len(self.tasks)

    def is_materialized(self):
        return False
    
    def get_output_keys(self) -> AbstractSet[str | bytes | int | float]:
        return set(self.tasks.keys())

# Define the tasks for each layer
tasks1 = {"output1": (lambda: "result1", ), "output2": (lambda: "result2", )}
tasks2 = {"output3": (lambda x: x + " processed", "output1"), "output4": (lambda x: x + " processed", "output2")}

# Create the layers
layer1 = CustomLayer("layer1", [], tasks1)
layer2 = CustomLayer("layer2", ["layer1"], tasks2)

# Create the high level graph
hlg = HighLevelGraph(
    layers={layer1.name: layer1, layer2.name: layer2},
    dependencies={layer2.name: {layer1.name}, layer1.name: set()},
)

In [40]:
client.get(hlg, ["output4", "output3"])

['result2 processed', 'result1 processed']